# Mask R-CNN for Keypoint Detection

Example showing how to do keypoint detection with Mask R-CNN

In [1]:
USE_GPU    = True
MODEL_NAME = "run2"

%matplotlib inline
import importlib
import os

if not USE_GPU:
    os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

import sys
import random
import math
import re
import time
import numpy as np
import cv2
import matplotlib
import matplotlib.pyplot as plt
import imgaug
import time

# Root directory of the project
ROOT_DIR = os.path.abspath(".")

# Import Mask RCNN
sys.path.append(ROOT_DIR)  # To find local version of the library

from mrcnn.config import Config
from mrcnn import utils
import mrcnn.model as modellib
from mrcnn import visualize
from mrcnn.model import log

# Directory to save logs and trained model
MODEL_DIR = os.path.join(ROOT_DIR, "logs", MODEL_NAME)

# Local path to trained weights file
COCO_MODEL_PATH = os.path.join(ROOT_DIR, "mask_rcnn_coco.h5")

# Download COCO trained weights from Releases if needed
if not os.path.exists(COCO_MODEL_PATH):
    utils.download_trained_weights(COCO_MODEL_PATH)

/opt/conda/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)
/opt/conda/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


## Load COCO Datasets for Keypoint Detection

In [2]:
from coco import coco_keypoints
importlib.reload(coco_keypoints)

# COCO dataset dir
COCO_DATA_DIR = "A:/Programming/DeepLearningDatasets/coco" if os.path.isdir("A:/Programming/DeepLearningDatasets/coco") else os.path.join(ROOT_DIR, "data/coco")

# Prepare dataset
dataset_train = coco_keypoints.CocoDataset()
dataset_train.load_coco(COCO_DATA_DIR, subset="train", year="2017", auto_download=True)
dataset_train.prepare()

dataset_val = coco_keypoints.CocoDataset()
dataset_val.load_coco(COCO_DATA_DIR, subset="val", year="2017", auto_download=True)
dataset_val.prepare()

Will use images in /datasets/home/78/378/cs252cas/Pose_RCNN/data/coco/train2017
Will use annotations in /datasets/home/78/378/cs252cas/Pose_RCNN/data/coco/annotations/person_keypoints_train2017.json
loading annotations into memory...
Done (t=9.42s)
creating index...
index created!
Will use images in /datasets/home/78/378/cs252cas/Pose_RCNN/data/coco/val2017
Will use annotations in /datasets/home/78/378/cs252cas/Pose_RCNN/data/coco/annotations/person_keypoints_val2017.json
loading annotations into memory...
Done (t=0.28s)
creating index...
index created!


### Setup Training

In [3]:
class TrainConfig(Config):
    NAME = "coco_keypoints"
    
    # Batch size
    IMAGES_PER_GPU = 2
    GPU_COUNT = 1
    #MAX_GT_INSTANCES = 1
    
    # We use resnet50
    BACKBONE = "resnet50"
    
    # Set number of classes
    NUM_CLASSES   = dataset_train.num_classes
    NUM_KEYPOINTS = dataset_train.num_kp_classes
    
    # DEBUG:
    STEPS_PER_EPOCH  = 1000 if USE_GPU else 10
    VALIDATION_STEPS = 50 if USE_GPU else 5
    
    TRAIN_ROIS_PER_IMAGE = 512
    
    RPN_NMS_THRESHOLD = 0.5
    
    IMAGE_MIN_DIM = 256
    IMAGE_MAX_DIM = 256

config = TrainConfig()
config.display()


Configurations:
BACKBONE                       resnet50
BACKBONE_STRIDES               [4, 8, 16, 32, 64]
BATCH_SIZE                     2
BBOX_STD_DEV                   [0.1 0.1 0.2 0.2]
DETECTION_MAX_INSTANCES        100
DETECTION_MIN_CONFIDENCE       0.7
DETECTION_NMS_THRESHOLD        0.3
GPU_COUNT                      1
GRADIENT_CLIP_NORM             5.0
IMAGES_PER_GPU                 2
IMAGE_MAX_DIM                  256
IMAGE_META_SIZE                14
IMAGE_MIN_DIM                  256
IMAGE_MIN_SCALE                0
IMAGE_RESIZE_MODE              square
IMAGE_SHAPE                    [256 256   3]
LEARNING_MOMENTUM              0.9
LEARNING_RATE                  0.001
LOSS_WEIGHTS                   {'rpn_class_loss': 1.0, 'rpn_bbox_loss': 1.0, 'mrcnn_class_loss': 1.0, 'mrcnn_bbox_loss': 1.0, 'mrcnn_mask_loss': 1.0, 'mrcnn_kp_mask_loss': 1.0}
MASK_POOL_SIZE                 14
MASK_SHAPE                     [28, 28]
MAX_GT_INSTANCES               100
MEAN_PIXEL                 

## Train

In [4]:
if TRAIN:
    importlib.reload(modellib)

    # Create model
    print("Creating model...")
    model = modellib.MaskRCNN(mode="training", model_dir=MODEL_DIR, config=config)

    # Get path to saved weights
    try:
        model_path = model.find_last()[1]
        if model_path is None: raise Exception("")
            
        # Load trained weights
        print("Loading weights from ", model_path)
        model.load_weights(model_path, by_name=True)
    except:
        # Load weights trained on MS-COCO
        print("Loading weights pretrained on MS-COCO")
        model.load_weights(COCO_MODEL_PATH, by_name=True,
                           exclude=["mrcnn_class_logits", "mrcnn_bbox_fc", "mrcnn_bbox", "mrcnn_mask"])

Creating model...
gt_kp_masks.shape (?, 17)
kp_masks.shape (?, 17)
kps_1d_indices.shape (?,)
kps_x.shape (?, ?)
kps_x.shape (?, ?)
kps_x.shape (?, ?)
masks.shape (?, 28, 28)
gt_kp_masks.shape (?, 17)
kp_masks.shape (?, 17)
kps_1d_indices.shape (?,)
kps_x.shape (?, ?)
kps_x.shape (?, ?)
kps_x.shape (?, ?)
masks.shape (?, 28, 28)
mrcnn_kp_masks.shape (?, 512, 17, 28, 28)
Loading weights from  /datasets/home/78/378/cs252cas/Pose_RCNN/logs/run_1/coco_keypoints20180606T2342/mask_rcnn_coco_keypoints_0012.h5


In [6]:
if TRAIN:
    importlib.reload(modellib)
    
    train_start_time = time.time()
    
    # Training - Stage 1
    print("Training network heads...")
    model.train(dataset_train, dataset_val,
                learning_rate=config.LEARNING_RATE,
                epochs=10,
                layers="heads",
                augmentation=None)

Training network heads...

Starting at epoch 0. LR=0.001

Checkpoint Path: /datasets/home/78/378/cs252cas/Pose_RCNN/logs/run_1/coco_keypoints20180606T2342/mask_rcnn_coco_keypoints_{epoch:04d}.h5
Selecting layers to train
fpn_c5p5               (Conv2D)
fpn_c4p4               (Conv2D)
fpn_c3p3               (Conv2D)
fpn_c2p2               (Conv2D)
fpn_p5                 (Conv2D)
fpn_p2                 (Conv2D)
fpn_p3                 (Conv2D)
fpn_p4                 (Conv2D)
In model:  rpn_model
    rpn_conv_shared        (Conv2D)
    rpn_class_raw          (Conv2D)
    rpn_bbox_pred          (Conv2D)
mrcnn_kp_mask_conv0    (TimeDistributed)
mrcnn_kp_mask_bn0      (TimeDistributed)
mrcnn_kp_mask_conv1    (TimeDistributed)
mrcnn_kp_mask_bn1      (TimeDistributed)
mrcnn_kp_mask_conv2    (TimeDistributed)
mrcnn_kp_mask_bn2      (TimeDistributed)
mrcnn_kp_mask_conv3    (TimeDistributed)
mrcnn_kp_mask_bn3      (TimeDistributed)
mrcnn_kp_mask_conv4    (TimeDistributed)
mrcnn_mask_conv1       (T

/opt/conda/lib/python3.6/site-packages/tensorflow/python/ops/gradients_impl.py:96: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
/opt/conda/lib/python3.6/site-packages/keras/engine/training.py:2023: UserWarning: Using a generator with `use_multiprocessing=True` and multiple workers may duplicate your data. Please consider using the`keras.utils.Sequence class.
  UserWarning('Using a generator with `use_multiprocessing=True`'


Epoch 1/10
 999/1000 [============================>.] - ETA: 1s - loss: 7.6344 - rpn_class_loss: 0.0450 - rpn_bbox_loss: 0.6008 - mrcnn_class_loss: 0.0119 - mrcnn_bbox_loss: 0.4346 - mrcnn_mask_loss: 0.3921 - mrcnn_kp_mask_loss: 6.1501

/opt/conda/lib/python3.6/site-packages/keras/engine/training.py:2251: UserWarning: Using a generator with `use_multiprocessing=True` and multiple workers may duplicate your data. Please consider using the`keras.utils.Sequence class.
  UserWarning('Using a generator with `use_multiprocessing=True`'


1000/1000 [==============================] - 1360s 1s/step - loss: 7.6340 - rpn_class_loss: 0.0450 - rpn_bbox_loss: 0.6006 - mrcnn_class_loss: 0.0119 - mrcnn_bbox_loss: 0.4348 - mrcnn_mask_loss: 0.3920 - mrcnn_kp_mask_loss: 6.1497 - val_loss: 7.1250 - val_rpn_class_loss: 0.0339 - val_rpn_bbox_loss: 0.5319 - val_mrcnn_class_loss: 0.0094 - val_mrcnn_bbox_loss: 0.3700 - val_mrcnn_mask_loss: 0.3724 - val_mrcnn_kp_mask_loss: 5.8074
Epoch 2/10
1000/1000 [==============================] - 1348s 1s/step - loss: 6.9487 - rpn_class_loss: 0.0423 - rpn_bbox_loss: 0.5566 - mrcnn_class_loss: 0.0102 - mrcnn_bbox_loss: 0.3699 - mrcnn_mask_loss: 0.3736 - mrcnn_kp_mask_loss: 5.5961 - val_loss: 6.6466 - val_rpn_class_loss: 0.0343 - val_rpn_bbox_loss: 0.4660 - val_mrcnn_class_loss: 0.0089 - val_mrcnn_bbox_loss: 0.3637 - val_mrcnn_mask_loss: 0.3703 - val_mrcnn_kp_mask_loss: 5.4034
Epoch 3/10
1000/1000 [==============================] - 1365s 1s/step - loss: 6.6446 - rpn_class_loss: 0.0381 - rpn_bbox_loss: 

In [7]:
if TRAIN:
    importlib.reload(modellib)

    # Training - Stage 2
    # Finetune layers from ResNet stage 4 and up
    print("Fine tune Resnet stage 4 and up...")
    model.train(dataset_train, dataset_val,
                learning_rate=config.LEARNING_RATE,
                epochs=10,
                layers="4+",
                augmentation=None)

Fine tune Resnet stage 4 and up...

Starting at epoch 10. LR=0.001

Checkpoint Path: /datasets/home/78/378/cs252cas/Pose_RCNN/logs/run_1/coco_keypoints20180606T2342/mask_rcnn_coco_keypoints_{epoch:04d}.h5
Selecting layers to train
res4a_branch2a         (Conv2D)
bn4a_branch2a          (BatchNorm)
res4a_branch2b         (Conv2D)
bn4a_branch2b          (BatchNorm)
res4a_branch2c         (Conv2D)
res4a_branch1          (Conv2D)
bn4a_branch2c          (BatchNorm)
bn4a_branch1           (BatchNorm)
res4b_branch2a         (Conv2D)
bn4b_branch2a          (BatchNorm)
res4b_branch2b         (Conv2D)
bn4b_branch2b          (BatchNorm)
res4b_branch2c         (Conv2D)
bn4b_branch2c          (BatchNorm)
res4c_branch2a         (Conv2D)
bn4c_branch2a          (BatchNorm)
res4c_branch2b         (Conv2D)
bn4c_branch2b          (BatchNorm)
res4c_branch2c         (Conv2D)
bn4c_branch2c          (BatchNorm)
res4d_branch2a         (Conv2D)
bn4d_branch2a          (BatchNorm)
res4d_branch2b         (Conv2D)


/opt/conda/lib/python3.6/site-packages/tensorflow/python/ops/gradients_impl.py:96: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
/opt/conda/lib/python3.6/site-packages/keras/engine/training.py:2023: UserWarning: Using a generator with `use_multiprocessing=True` and multiple workers may duplicate your data. Please consider using the`keras.utils.Sequence class.
  UserWarning('Using a generator with `use_multiprocessing=True`'


In [5]:
if TRAIN:
    importlib.reload(modellib)

    # Training - Stage 3
    # Fine tune all layers
    print("Fine tune all layers...")
    model.train(dataset_train, dataset_val,
                learning_rate=config.LEARNING_RATE / 10,
                epochs=20,
                layers="all",
                augmentation=None)
    
    # Training done
    print("Training done")

Fine tune all layers...

Starting at epoch 12. LR=0.0001

Checkpoint Path: /datasets/home/78/378/cs252cas/Pose_RCNN/logs/run_1/coco_keypoints20180606T2342/mask_rcnn_coco_keypoints_{epoch:04d}.h5
Selecting layers to train
conv1                  (Conv2D)
bn_conv1               (BatchNorm)
res2a_branch2a         (Conv2D)
bn2a_branch2a          (BatchNorm)
res2a_branch2b         (Conv2D)
bn2a_branch2b          (BatchNorm)
res2a_branch2c         (Conv2D)
res2a_branch1          (Conv2D)
bn2a_branch2c          (BatchNorm)
bn2a_branch1           (BatchNorm)
res2b_branch2a         (Conv2D)
bn2b_branch2a          (BatchNorm)
res2b_branch2b         (Conv2D)
bn2b_branch2b          (BatchNorm)
res2b_branch2c         (Conv2D)
bn2b_branch2c          (BatchNorm)
res2c_branch2a         (Conv2D)
bn2c_branch2a          (BatchNorm)
res2c_branch2b         (Conv2D)
bn2c_branch2b          (BatchNorm)
res2c_branch2c         (Conv2D)
bn2c_branch2c          (BatchNorm)
res3a_branch2a         (Conv2D)
bn3a_branc

/opt/conda/lib/python3.6/site-packages/tensorflow/python/ops/gradients_impl.py:96: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
/opt/conda/lib/python3.6/site-packages/keras/engine/training.py:2023: UserWarning: Using a generator with `use_multiprocessing=True` and multiple workers may duplicate your data. Please consider using the`keras.utils.Sequence class.
  UserWarning('Using a generator with `use_multiprocessing=True`'


Epoch 13/20
 999/1000 [============================>.] - ETA: 1s - loss: 5.7335 - rpn_class_loss: 0.0323 - rpn_bbox_loss: 0.4723 - mrcnn_class_loss: 0.0080 - mrcnn_bbox_loss: 0.3007 - mrcnn_mask_loss: 0.3393 - mrcnn_kp_mask_loss: 4.5808

/opt/conda/lib/python3.6/site-packages/keras/engine/training.py:2251: UserWarning: Using a generator with `use_multiprocessing=True` and multiple workers may duplicate your data. Please consider using the`keras.utils.Sequence class.
  UserWarning('Using a generator with `use_multiprocessing=True`'


1000/1000 [==============================] - 1430s 1s/step - loss: 5.7326 - rpn_class_loss: 0.0322 - rpn_bbox_loss: 0.4720 - mrcnn_class_loss: 0.0080 - mrcnn_bbox_loss: 0.3005 - mrcnn_mask_loss: 0.3391 - mrcnn_kp_mask_loss: 4.5806 - val_loss: 5.8549 - val_rpn_class_loss: 0.0309 - val_rpn_bbox_loss: 0.4352 - val_mrcnn_class_loss: 0.0085 - val_mrcnn_bbox_loss: 0.3137 - val_mrcnn_mask_loss: 0.3375 - val_mrcnn_kp_mask_loss: 4.7291
Epoch 14/20
1000/1000 [==============================] - 1448s 1s/step - loss: 5.7510 - rpn_class_loss: 0.0322 - rpn_bbox_loss: 0.4693 - mrcnn_class_loss: 0.0082 - mrcnn_bbox_loss: 0.3006 - mrcnn_mask_loss: 0.3401 - mrcnn_kp_mask_loss: 4.6005 - val_loss: 5.8035 - val_rpn_class_loss: 0.0272 - val_rpn_bbox_loss: 0.3643 - val_mrcnn_class_loss: 0.0086 - val_mrcnn_bbox_loss: 0.3090 - val_mrcnn_mask_loss: 0.3566 - val_mrcnn_kp_mask_loss: 4.7378
Epoch 15/20
1000/1000 [==============================] - 1395s 1s/step - loss: 5.6580 - rpn_class_loss: 0.0298 - rpn_bbox_loss

Explanation of losses:

- rpn_class_loss: "How well does the network separate positive from negative anchors?"
- rpn_bbox_loss: "How accurate are proposed the bounding boxes?"
- mrcnn_class_loss: "How well does the network distinguish people from background RoIs?" (really, this is the same as 1.)
- mrcnn_bbox_loss: "Bounding box refinement loss" (is this the same as 2.?)
- mrcnn_mask_loss: "How accurate are the predicted keypoints?" (per mask softmax cross-entropy loss)